Vulnerability of an Area to Serious Volcanic Eruptions	dis.008	http://sedac.ciesin.columbia.edu/data/set/ndh-volcano-hazard-frequency-distribution

dis.008 http://sedac.ciesin.columbia.edu/data/set/ndh-volcano-hazard-frequency-distribution
Downloaded to RW_Data/Rasters/gdvol/
File: gdvol.asc 

Import libraries

In [1]:
# Libraries for downloading data from remote server (may be ftp)
import requests
from urllib.request import urlopen
from contextlib import closing
import shutil

# Library for uploading/downloading data to/from S3
import boto3

# Libraries for handling data
import rasterio as rio
import numpy as np
# from netCDF4 import Dataset
# import pandas as pd
# import scipy

# Libraries for various helper functions
# from datetime import datetime
import os
import threading
import sys
from glob import glob

s3 tools

In [2]:
s3_upload = boto3.client("s3")
s3_download = boto3.resource("s3")

s3_bucket = "wri-public-data"
s3_folder = "resourcewatch/raster/dis_008_vulnerability_to_volcanic_eruptions/"

s3_file = "dis_008_vulnerability_to_volcanic_eruptions.asc"

s3_key_orig = s3_folder + s3_file
s3_key_edit = s3_key_orig[0:-4] + "_edit.tif"


class ProgressPercentage(object):
        def __init__(self, filename):
            self._filename = filename
            self._size = float(os.path.getsize(filename))
            self._seen_so_far = 0
            self._lock = threading.Lock()

        def __call__(self, bytes_amount):
            # To simplify we'll assume this is hooked up
            # to a single filename.
            with self._lock:
                self._seen_so_far += bytes_amount
                percentage = (self._seen_so_far / self._size) * 100
                sys.stdout.write("\r%s  %s / %s  (%.2f%%)"%(
                        self._filename, self._seen_so_far, self._size,
                        percentage))
                sys.stdout.flush()

Define local file locations

In [3]:
local_folder = "/Users/nathansuberi/Desktop/RW_Data/Rasters/gdvol/"
file_name = "gdvol.asc"

local_orig = local_folder + file_name

orig_extension_length = 4 #4 for each char in .tif
local_edit = local_orig[:-orig_extension_length] + "edit.tif" 

Use rasterio to reproject and compress

In [4]:
with rio.open(local_orig, 'r') as src:
    print(src.profile)
    data = src.read()
    
print(data)

{'driver': 'AAIGrid', 'dtype': 'int32', 'nodata': -9999.0, 'width': 8640, 'height': 3432, 'count': 1, 'crs': CRS({'init': 'epsg:4326'}), 'transform': (-180.0, 0.041666666666666664, 0.0, 84.984928, 0.0, -0.041666666666666664), 'affine': Affine(0.041666666666666664, 0.0, -180.0,
       0.0, -0.041666666666666664, 84.984928), 'tiled': False}
[[[-9999 -9999 -9999 ..., -9999 -9999 -9999]
  [-9999 -9999 -9999 ..., -9999 -9999 -9999]
  [-9999 -9999 -9999 ..., -9999 -9999 -9999]
  ..., 
  [-9999 -9999 -9999 ..., -9999 -9999 -9999]
  [-9999 -9999 -9999 ..., -9999 -9999 -9999]
  [-9999 -9999 -9999 ..., -9999 -9999 -9999]]]


In [6]:
# Note - this is the core of Vizz's netcdf2tif function

with rio.open(local_orig, 'r') as src:
    # This assumes data is readable by rasterio
    # May need to open instead with netcdf4.Dataset, for example
   
    data = src.read()[0]
    
    rows = data.shape[0]
    columns = data.shape[1]
    
    print(rows)
    print(columns)
    
    # Latitude bounds
    south_lat = -90
    north_lat = 90

    # Longitude bounds
    west_lon = -180
    east_lon = 180
    
    transform = rio.transform.from_bounds(west_lon, south_lat, east_lon, north_lat, columns, rows)
    
    # Profile
    
    no_data_val = -9999
    target_projection = 'EPSG:4326'
    target_data_type = np.int32
    
    profile = {
        'driver':'GTiff', 
        'height':rows, 
        'width':columns, 
        'count':1, 
        'dtype':target_data_type, 
        'crs':target_projection, 
        'transform':transform, 
        'compress':'lzw', 
        'nodata': no_data_val
    }
    
    with rio.open(local_edit, "w", **profile) as dst:
        dst.write(data.astype(profile["dtype"]), 1)

3432
8640


Upload orig and edit files to s3

In [7]:
# Original
s3_upload.upload_file(local_orig, s3_bucket, s3_key_orig,
                         Callback=ProgressPercentage(local_orig))

# Edit
s3_upload.upload_file(local_edit, s3_bucket, s3_key_edit,
                         Callback=ProgressPercentage(local_edit))

/Users/nathansuberi/Desktop/RW_Data/Rasters/gdvol/gdvoledit.tif  2217577 / 2217577.0  (100.00%)